In [1]:
import smtplib

In [2]:
from email.mime.text import MIMEText

In [3]:
# 시스템 라이브러리 활용 패키지
import os

# 이메일 메시지에 형식을 활용하기 위한 패키지
from email.mime.multipart import MIMEMultipart

# 이메일 메시지 프로토콜 처리기(엔코더)
from email import encoders

# 텍스트 형식 메일
from email.mime.text import MIMEText

# 이미지 형식 메일
from email.mime.image import MIMEImage

# 오디오 형식 메일
from email.mime.audio import MIMEAudio

# 이 모든 정보들을 관장하는 객체
from email.mime.base import MIMEBase

In [4]:
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info["smtp_server"], smtp_info["smtp_port"]) as server:
        # TLS 보안 설정 및 시작 (보안 정책상 필수)
        server.starttls()
        
        # 로그인
        server.login(smtp_info["smtp_user_id"], smtp_info["smtp_user_pw"])
        
        # 이메일 전송
        # 메시지 전송시 반드시 as_string()으로 변환하여 전송해야함
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공!!!')
        else:
            print(res)

In [5]:
def make_multimsg(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        # text 형식인 경우
        if key == 'text':
            with open(value['filename'], encoding='utf-8') as fp:
                msg = MIMEText(fp.read(), _subtype = value['subtype'])
        # image 형식인 경우
        elif key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
        # audio, video는 별도 설정 없이 진행하도록 함
        elif key == 'audio':
            print("음원은 지원하지 않습니다!")
        elif key == 'video':
            print("영상은 지원하지 않습니다!")
        # 그 외의 형식
        else:
            with open(value['filename'], 'rb') as fp:
                msg = MIMEBase(value['maintype'], _subtype = value['subtype'])
                msg.set_payload(fp.read())
                encoders.encode_base64(msg)
                
        # 파일 이름은 첨부파일 제목
        msg.add_header('Content-Disposition', 'attachment', 
                       filename = os.path.basename(value['filename']))

        # 파일 첨부!
        multi.attach(msg)
        
    return multi

In [6]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json
import requests
import datetime
import os

In [ ]:
app = Flask(__name__)
CORS(app)
@app.route('/sendNaverEamil', methods = ['GET', 'POST'])
def sendNaverEamil():
    params = json.loads(request.get_data(), encoding='utf-8')
    print("넘어온값: ", params)
    #넘어온값이 없으면
    if len(params) == 0:
        return jsonify("No Parameter")
    
    params_number= ""
    Authentication_num = ""
    values = []
    for key in params.keys():
        values.append(params[key])
    
    
    Authentication_num = values[0]
    print("인증번호:", Authentication_num)
    
    smtp_info = dict({
    "smtp_server": "smtp.naver.com",
    "smtp_user_id": "",
    "smtp_user_pw": "",
    "smtp_port": 587
    })

    title = "[EVS]인증번호"
    content =  str(values[0])
    sender = smtp_info['smtp_user_id']
    receiver = smtp_info['smtp_user_id']

    msg = MIMEText(_text = content, _charset = "utf-8")

    msg['Subject'] = title
    msg['From'] = sender
    msg['To'] = receiver
    
    send_email(smtp_info, msg)
    
    
    
    return params_number
    
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2021 22:53:16] "OPTIONS /sendNaverEamil HTTP/1.1" 200 -
<ipython-input-8-a7b3c4db7915>:5: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  params = json.loads(request.get_data(), encoding='utf-8')


넘어온값:  {'randomNum2': 46601}
인증번호: 46601


127.0.0.1 - - [09/Nov/2021 22:53:17] "POST /sendNaverEamil HTTP/1.1" 200 -


이메일 전송 성공!!!
